In [7]:
from pyspark import RDD
from pyspark import SparkContext
from operator import add


In [19]:
def map_rdd_tags():
    genre_rdd=sc.pickleFile("../data/metadata-artist_terms/")
    year_rdd=sc.pickleFile("../data/musicbrainz-songs/")
    metadata_rdd=sc.pickleFile('../data/metadata-songs/')

    f=genre_rdd.map(lambda x: (x[0],x[1]))

    year=year_rdd.map(lambda x: (x[0],x[1]['year'][0]))

    hotness=metadata_rdd.map(lambda x: (x[0],x[1]['song_hotttnesss'][0]))
    joined=f.join(year).join(hotness)
    by=joined.map(lambda x: (x[1][0][1],x[1][0][0],x[1][1])) #year,array tags, hotness
    return by

In [22]:
full=map_rdd_tags()

In [49]:
s=full.collect()
s[0]

(1992, array(['grunge', 'alternative metal', 'heavy metal', 'hard rock',
        'alternative rock', 'rock', 'seattle', 'alternative pop rock',
        'soundtrack', 'metal', 'heavy', 'acoustic', 'lyrics', 'alternative',
        'classic', 'beautiful', 'soul', 'sludge', 'guitar', 'mellow',
        'epic', 'aggressive', 'high school'],
       dtype='|S256'), 0.81839437971637685)

In [25]:
def filter_year(rdd,year):
    return rdd.filter(lambda x:x[0]==year)

In [44]:
def count_tags(rdd):
    m=rdd.flatMap(lambda x: [i for i in x[1]]).map(lambda x : (x,1)).reduceByKey(add)
    c=m.collect()
    return sorted(c,key=lambda x:x[1],reverse=True)

In [47]:
f=filter_year(full,2000)
c=count_tags(f)
c[:10]

[('rock', 136),
 ('pop', 114),
 ('electronic', 98),
 ('jazz', 66),
 ('united states', 66),
 ('hip hop', 66),
 ('alternative', 66),
 ('soul', 58),
 ('folk', 53),
 ('indie', 51)]

In [34]:
def filter_hotness(rdd,hotness):
    return rdd.filter(lambda x : x[2]>=hotness)

In [48]:

ht=filter_hotness(f,0.7)
c=count_tags(ht)
c[:10]

[('rock', 9),
 ('pop', 6),
 ('punk', 5),
 ('rap', 5),
 ('soundtrack', 5),
 ('hip hop', 5),
 ('alternative', 5),
 ('club', 4),
 ('indie', 4),
 ('guitar', 4)]